In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy import *
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy
from scipy import spatial
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOP_WORDS = stopwords.words("english")
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve, auc, roc_curve ,f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Capstone Data/train.csv") # importing the dataset

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

In [ ]:
df.head(20)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df7 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
print(df7.head())

In [ ]:
print(vectors)

In [ ]:
print(feature_names)

In [ ]:
df['question1']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
question = list(df['question1']) + list(df['question2'])

questions = [ques for ques in question if ques not in STOP_WORDS]

tfidf = TfidfVectorizer(lowercase=False, )
x=tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
print(x) # first id , feautre_names no. , tfidf score

In [ ]:
tfidf.idf_

In [ ]:
abf = tfidf.get_feature_names()
abf[10000:10020]

In [ ]:
word2tfidf['what']

In [ ]:
len(abf)==len(word2tfidf)

In [ ]:
d = documentA.split(' ')
print(d)

l = [word for word in d if word not in STOP_WORDS]
print(l)

In [ ]:
for i in l:
  print("{0}={1}".format(i,word2tfidf[i]))

## Xgboost

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

xgb_model = xgb.XGBClassifier(n_estimators=150,random_state=42, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,xgb_prediction))
print(classification_report(y_valid, xgb_prediction))

In [ ]:
import pickle
filename = 'finalized_model1.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

In [ ]:
X_valid.shape

In [ ]:
print(X_valid[0].shape)

In [ ]:
filename = 'finalized_model1.sav'
# test=X_valid[105000]
result=loaded_model.predict(test)[0]
print(result)

In [ ]:
filename = 'finalized_model1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_valid)

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())

In [ ]:
question = list(df['question2']) + list(df['question1']) 
print(question[0])

In [ ]:
abc = tfidf_vect.transform(["Who would be better for India, Donald Trump or Hillary Clinton?"])
for i in tqdm(question):
  adc = tfidf_vect.transform([i])
  x=scipy.sparse.hstack((abc,adc))
  if loaded_model.predict(x)==1:
    print(i)

In [ ]:
abc = tfidf_vect.transform(["How can i grow my muscles"])
adc = tfidf_vect.transform([""])
x=scipy.sparse.hstack((abc,adc))

In [ ]:
print(abc)
print(abc.shape)
print(adc.shape)

In [ ]:
x=scipy.sparse.hstack((abc,adc))
print(x.shape)

In [ ]:
result=loaded_model.predict(x)
print(result)

## Random Forest

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

clf = RandomForestClassifier(n_estimators=80).fit(X_train,y_train)
adb = clf.predict(X_valid)
print('word level tf-idf validation score:', f1_score(y_valid, clf.predict(X_valid), average='macro'))